In [1]:
import cv2 as cv
import numpy as np 
import tensorflow as tf

In [2]:
model_emotion = tf.keras.models.load_model('artifacts/emotion_model.h5')
model_emotion.compile(
                optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[
                        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall'),
                        tf.keras.metrics.AUC(name='auc')
                        ]
                )

model_ptsd = tf.keras.models.load_model('artifacts/ptsd_model.h5')
model_ptsd.compile(
                optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=[
                        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall'),
                        tf.keras.metrics.AUC(name='auc')
                        ]
                )

### TESS

In [4]:
def get_spectrogram(waveform):
    spectrogram = tf.signal.stft(
                                waveform, 
                                frame_length=255, 
                                frame_step=128
                                )
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

def inference_emotion(
                    audio_file,
                    class_dict = {
                                0: 'Angry',
                                1: 'Fear',
                                2: 'Happy',
                                3: 'Neutral',
                                4: 'Pleasant_surprise',
                                5: 'Sad',
                                6: 'disgust'
                                },
                    output_sequence_length=16000
                    ):
    raw_audio = tf.io.read_file(audio_file)
    waveform = tf.audio.decode_wav(raw_audio).audio
    waveform = tf.squeeze(waveform, axis=-1)
    waveform_length = tf.shape(waveform)[0]
    if waveform_length < output_sequence_length:
        waveform = tf.concat([waveform, tf.zeros(output_sequence_length - waveform_length)], 0)
    elif waveform_length > output_sequence_length:
        waveform = waveform[:output_sequence_length]
    spectrogram = get_spectrogram(waveform)
    spectrogram = tf.expand_dims(spectrogram, axis=0)
    
    prediction = model_emotion.predict(spectrogram)
    prediction = tf.nn.softmax(prediction)
    prediction = tf.squeeze(prediction, axis=0)
    prediction = prediction.numpy().argmax()
    prediction = class_dict[prediction]
    return prediction

In [8]:
inference_emotion('data/TESS/Neutral/1001_IEO_NEU_XX.wav')

1/1 [==============================] - 0s 28ms/step


'Neutral'

### PTSD

In [12]:
def preprocessing_function(image):
    return tf.keras.applications.xception.preprocess_input(image)
    
def inference_ptsd(
                    image_path,
                    class_dict = {
                                0: 'High Level', 
                                1: 'Low Level', 
                                2: 'Medium Level'
                                }
                    ):
    image = cv.imread(image_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image = cv.resize(image, (72, 72))
    image = np.expand_dims(image, axis=0)
    image = preprocessing_function(image)
    pred = model_ptsd.predict(
                            image,
                            verbose=0
                            ).squeeze()
    label = int(pred.argmax())
    label = class_dict[label]
    return label

In [13]:
inference_ptsd('data/PTSD/High Level/Training_3908.jpg')

'High Level'